In [ ]:
""" Trains an agent with (stochastic) Policy Gradients on Pong. Uses OpenAI Gym. """
import numpy as np
import _pickle as pickle
import gym

# hyperparameters
H = 200 # number of hidden layer neurons
batch_size = 10 # every how many episodes to do a param update?
learning_rate = 1e-4
gamma = 0.99 # discount factor for reward
decay_rate = 0.99 # decay factor for RMSProp leaky sum of grad^2
resume = False # resume from previous checkpoint?
render = False

# model initialization
D = 80 * 80 # input dimensionality: 80x80 grid
if resume:
    model = pickle.load(open('save.p', 'rb'))
else:
    model = {}
    model['W1'] = np.random.randn(H,D) / np.sqrt(D) # "Xavier" initialization
    model['W2'] = np.random.randn(H) / np.sqrt(H)

grad_buffer = { k : np.zeros_like(v) for k,v in model.items() } # update buffers that add up gradients over a batch
rmsprop_cache = { k : np.zeros_like(v) for k,v in model.items() } # rmsprop memory

def sigmoid(x): 
    return 1.0 / (1.0 + np.exp(-x)) # sigmoid "squashing" function to interval [0,1]

def prepro(I):
    """ prepro 210x160x3 uint8 frame into 6400 (80x80) 1D float vector """
    I = I[35:195] # crop
    I = I[::2,::2,0] # downsample by factor of 2
    I[I == 144] = 0 # erase background (background type 1)
    I[I == 109] = 0 # erase background (background type 2)
    I[I != 0] = 1 # everything else (paddles, ball) just set to 1
    return I.astype(np.float).ravel()

def discount_rewards(r):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(range(0, r.size)):
        if r[t] != 0: running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r

def policy_forward(x):
    h = np.dot(model['W1'], x)
    h[h<0] = 0 # ReLU nonlinearity
    logp = np.dot(model['W2'], h)
    p = sigmoid(logp)
    return p, h # return probability of taking action 2, and hidden state

def policy_backward(eph, epdlogp):
    """ backward pass. (eph is array of intermediate hidden states) """
    dW2 = np.dot(eph.T, epdlogp).ravel()
    dh = np.outer(epdlogp, model['W2'])
    dh[eph <= 0] = 0 # backpro prelu
    dW1 = np.dot(dh.T, epx)
    return {'W1':dW1, 'W2':dW2}

env = gym.make("Pong-v0")
observation = env.reset()
prev_x = None # used in computing the difference frame
xs,hs,dlogps,drs = [],[],[],[]
running_reward = None
reward_sum = 0
episode_number = 0

while True:
    if render: env.render()

    # preprocess the observation, set input to network to be difference image
    cur_x = prepro(observation)
    x = cur_x - prev_x if prev_x is not None else np.zeros(D)
    prev_x = cur_x
    
    # forward the policy network and sample an action from the returned probability
    aprob, h = policy_forward(x)
    action = 2 if np.random.uniform() < aprob else 3 # roll the dice!

    # record various intermediates (needed later for backprop)
    xs.append(x) # observation
    hs.append(h) # hidden state
    y = 1 if action == 2 else 0 # a "fake label"
    dlogps.append(y - aprob) # grad that encourages the action that was taken to be taken (see http://cs231n.github.io/neural-networks-2/#losses if confused)

    # step the environment and get new measurements
    observation, reward, done, info = env.step(action)
    reward_sum += reward

    drs.append(reward) # record reward (has to be done after we call step() to get reward for previous action)

    if done: # an episode finished
        episode_number += 1

        # stack together all inputs, hidden states, action gradients, and rewards for this episode
        epx = np.vstack(xs)
        eph = np.vstack(hs)
        epdlogp = np.vstack(dlogps)
        epr = np.vstack(drs)
        xs,hs,dlogps,drs = [],[],[],[] # reset array memory

        # compute the discounted reward backwards through time
        discounted_epr = discount_rewards(epr)
        # standardize the rewards to be unit normal (helps control the gradient estimator variance)
        discounted_epr -= np.mean(discounted_epr)
        discounted_epr /= np.std(discounted_epr)

        epdlogp *= discounted_epr # modulate the gradient with advantage (PG magic happens right here.)
        grad = policy_backward(eph, epdlogp)
        for k in model: grad_buffer[k] += grad[k] # accumulate grad over batch

        # perform rmsprop parameter update every batch_size episodes
        if episode_number % batch_size == 0:
            for k,v in model.items():
                g = grad_buffer[k] # gradient
                rmsprop_cache[k] = decay_rate * rmsprop_cache[k] + (1 - decay_rate) * g**2
                model[k] += learning_rate * g / (np.sqrt(rmsprop_cache[k]) + 1e-5)
                grad_buffer[k] = np.zeros_like(v) # reset batch gradient buffer
                
        # boring book-keeping
        running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
        print('resetting env. episode reward total was %f. running mean: %f' % (reward_sum, running_reward))
        if episode_number % 100 == 0: pickle.dump(model, open('save.p', 'wb'))
        reward_sum = 0
        observation = env.reset() # reset env
        prev_x = None

#     if reward != 0: # Pong has either +1 or -1 reward exactly when game ends.
#         print ('ep %d: game finished, reward: %f' % (episode_number, reward) + ('' if reward == -1 else ' !!!!!!!!'))

[2017-12-25 12:49:00,687] Making new env: Pong-v0


resetting env. episode reward total was -21.000000. running mean: -21.000000
resetting env. episode reward total was -20.000000. running mean: -20.990000
resetting env. episode reward total was -20.000000. running mean: -20.980100
resetting env. episode reward total was -20.000000. running mean: -20.970299
resetting env. episode reward total was -19.000000. running mean: -20.950596
resetting env. episode reward total was -21.000000. running mean: -20.951090
resetting env. episode reward total was -21.000000. running mean: -20.951579
resetting env. episode reward total was -21.000000. running mean: -20.952063
resetting env. episode reward total was -21.000000. running mean: -20.952543
resetting env. episode reward total was -21.000000. running mean: -20.953017
resetting env. episode reward total was -21.000000. running mean: -20.953487
resetting env. episode reward total was -21.000000. running mean: -20.953952
resetting env. episode reward total was -21.000000. running mean: -20.954413

resetting env. episode reward total was -20.000000. running mean: -20.620780
resetting env. episode reward total was -20.000000. running mean: -20.614572
resetting env. episode reward total was -21.000000. running mean: -20.618426
resetting env. episode reward total was -19.000000. running mean: -20.602242
resetting env. episode reward total was -21.000000. running mean: -20.606220
resetting env. episode reward total was -21.000000. running mean: -20.610157
resetting env. episode reward total was -21.000000. running mean: -20.614056
resetting env. episode reward total was -21.000000. running mean: -20.617915
resetting env. episode reward total was -21.000000. running mean: -20.621736
resetting env. episode reward total was -19.000000. running mean: -20.605519
resetting env. episode reward total was -21.000000. running mean: -20.609464
resetting env. episode reward total was -19.000000. running mean: -20.593369
resetting env. episode reward total was -20.000000. running mean: -20.587435

resetting env. episode reward total was -21.000000. running mean: -20.477520
resetting env. episode reward total was -21.000000. running mean: -20.482744
resetting env. episode reward total was -21.000000. running mean: -20.487917
resetting env. episode reward total was -21.000000. running mean: -20.493038
resetting env. episode reward total was -19.000000. running mean: -20.478107
resetting env. episode reward total was -21.000000. running mean: -20.483326
resetting env. episode reward total was -20.000000. running mean: -20.478493
resetting env. episode reward total was -21.000000. running mean: -20.483708
resetting env. episode reward total was -17.000000. running mean: -20.448871
resetting env. episode reward total was -21.000000. running mean: -20.454382
resetting env. episode reward total was -21.000000. running mean: -20.459839
resetting env. episode reward total was -21.000000. running mean: -20.465240
resetting env. episode reward total was -20.000000. running mean: -20.460588

resetting env. episode reward total was -21.000000. running mean: -20.360166
resetting env. episode reward total was -21.000000. running mean: -20.366564
resetting env. episode reward total was -19.000000. running mean: -20.352898
resetting env. episode reward total was -21.000000. running mean: -20.359369
resetting env. episode reward total was -21.000000. running mean: -20.365776
resetting env. episode reward total was -20.000000. running mean: -20.362118
resetting env. episode reward total was -20.000000. running mean: -20.358497
resetting env. episode reward total was -20.000000. running mean: -20.354912
resetting env. episode reward total was -20.000000. running mean: -20.351363
resetting env. episode reward total was -18.000000. running mean: -20.327849
resetting env. episode reward total was -21.000000. running mean: -20.334571
resetting env. episode reward total was -21.000000. running mean: -20.341225
resetting env. episode reward total was -21.000000. running mean: -20.347813

...

resetting env. episode reward total was -21.000000. running mean: -19.565986
resetting env. episode reward total was -21.000000. running mean: -19.580326
resetting env. episode reward total was -19.000000. running mean: -19.574523
resetting env. episode reward total was -20.000000. running mean: -19.578778
resetting env. episode reward total was -21.000000. running mean: -19.592990
resetting env. episode reward total was -17.000000. running mean: -19.567060
resetting env. episode reward total was -21.000000. running mean: -19.581390
resetting env. episode reward total was -20.000000. running mean: -19.585576
resetting env. episode reward total was -21.000000. running mean: -19.599720
resetting env. episode reward total was -19.000000. running mean: -19.593723
resetting env. episode reward total was -21.000000. running mean: -19.607785
resetting env. episode reward total was -18.000000. running mean: -19.591708
resetting env. episode reward total was -21.000000. running mean: -19.605790

resetting env. episode reward total was -19.000000. running mean: -19.497882
resetting env. episode reward total was -20.000000. running mean: -19.502903
resetting env. episode reward total was -20.000000. running mean: -19.507874
resetting env. episode reward total was -19.000000. running mean: -19.502795
resetting env. episode reward total was -20.000000. running mean: -19.507767
resetting env. episode reward total was -18.000000. running mean: -19.492690
resetting env. episode reward total was -15.000000. running mean: -19.447763
resetting env. episode reward total was -20.000000. running mean: -19.453285
resetting env. episode reward total was -19.000000. running mean: -19.448752
resetting env. episode reward total was -19.000000. running mean: -19.444265
resetting env. episode reward total was -18.000000. running mean: -19.429822
resetting env. episode reward total was -21.000000. running mean: -19.445524
resetting env. episode reward total was -18.000000. running mean: -19.431069

resetting env. episode reward total was -19.000000. running mean: -19.337612
resetting env. episode reward total was -19.000000. running mean: -19.334236
resetting env. episode reward total was -17.000000. running mean: -19.310894
resetting env. episode reward total was -17.000000. running mean: -19.287785
resetting env. episode reward total was -17.000000. running mean: -19.264907
resetting env. episode reward total was -21.000000. running mean: -19.282258
resetting env. episode reward total was -19.000000. running mean: -19.279435
resetting env. episode reward total was -21.000000. running mean: -19.296641
resetting env. episode reward total was -19.000000. running mean: -19.293674
resetting env. episode reward total was -21.000000. running mean: -19.310738
resetting env. episode reward total was -20.000000. running mean: -19.317630
resetting env. episode reward total was -19.000000. running mean: -19.314454
resetting env. episode reward total was -20.000000. running mean: -19.321309

resetting env. episode reward total was -19.000000. running mean: -19.355508
resetting env. episode reward total was -21.000000. running mean: -19.371953
resetting env. episode reward total was -18.000000. running mean: -19.358234
resetting env. episode reward total was -19.000000. running mean: -19.354651
resetting env. episode reward total was -21.000000. running mean: -19.371105
resetting env. episode reward total was -17.000000. running mean: -19.347394
resetting env. episode reward total was -19.000000. running mean: -19.343920
resetting env. episode reward total was -20.000000. running mean: -19.350481
resetting env. episode reward total was -21.000000. running mean: -19.366976
resetting env. episode reward total was -21.000000. running mean: -19.383306
resetting env. episode reward total was -21.000000. running mean: -19.399473
resetting env. episode reward total was -20.000000. running mean: -19.405478
resetting env. episode reward total was -20.000000. running mean: -19.411424

resetting env. episode reward total was -20.000000. running mean: -19.509931
resetting env. episode reward total was -20.000000. running mean: -19.514832
resetting env. episode reward total was -20.000000. running mean: -19.519683
resetting env. episode reward total was -19.000000. running mean: -19.514487
resetting env. episode reward total was -20.000000. running mean: -19.519342
resetting env. episode reward total was -20.000000. running mean: -19.524148
resetting env. episode reward total was -20.000000. running mean: -19.528907
resetting env. episode reward total was -18.000000. running mean: -19.513618
resetting env. episode reward total was -18.000000. running mean: -19.498482
resetting env. episode reward total was -18.000000. running mean: -19.483497
resetting env. episode reward total was -21.000000. running mean: -19.498662
resetting env. episode reward total was -17.000000. running mean: -19.473675
resetting env. episode reward total was -18.000000. running mean: -19.458938